In [18]:
import math
import csv
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import antropy as ant
from sklearn.feature_selection import SelectFdr, SelectPercentile, SelectKBest, chi2


In [4]:
begin, end = 1, 61 # (begin is inclusive, end is exclusive)
num_people = 14
count_samples = {
    "active": 8,
    "meditate": 8,
    "neutral": 8
}

class Sample:
    def __init__(self):
        self.data = {
            'RawEEG': [],
            'Alpha': [],
            'Low Beta': [],
            'High Beta': [],
            'Gamma': [],
            'Theta': [],
            'Delta': [],
            'Meditation': [],
            'Attention': []
        }

    def recordDataPoint(self, RawEEG, Attention, Meditation, Alpha, Delta, Theta, LowBeta, HighBeta, Gamma):
        self.data['RawEEG'].append(float(RawEEG))
        self.data['Attention'].append(float(Attention))
        self.data['Meditation'].append(float(Meditation))
        self.data['Alpha'].append(float(Alpha))
        self.data['Delta'].append(float(Delta))
        self.data['Theta'].append(float(Theta))
        self.data['Low Beta'].append(float(LowBeta))
        self.data['High Beta'].append(float(HighBeta))
        self.data['Gamma'].append(float(Gamma))

    '''
    Record a line of data from the CSV output, which takes form RawEEG, Alpha, Delta, Gamma, Low Beta, High Beta, Theta, Attention, Meditation

    '''
    def recordDataLine(self, line):
        self.recordDataPoint(line[0], line[7], line[8], line[1], line[2], line[6], line[4], line[5], line[3])
    
    def getEEG(self):
        return self.data['RawEEG']
    
    def getAttention(self):
        return self.data["Attention"]
    
    def getMeditation(self):
        return self.data["Meditation"]
    
    def getAlpha(self):
        return self.data["Alpha"]
    
    def getDelta(self):
        return self.data["Delta"]
    
    def getTheta(self):
        return self.data["Theta"]
    
    def getLowBeta(self):
        return self.data["Low Beta"]
    
    def getHighBeta(self):
        return self.data["High Beta"]
    
    def getGamma(self):
        return self.data["Gamma"]

    def get(self, key):
        return self.data[key]

    '''
    Filter out all outliers, as defined by being outside 3*std from the mean, and replace with mean of the samples around them
    '''
    def filter_outliers(self):
        sampleBad = False
        for key in ['RawEEG', 'Alpha', 'Theta', 'Low Beta', 'High Beta', "Gamma", 'Delta']:
            data = self.data[key]
            
            filtered = []
            for i, x in enumerate(data):
                iqr = np.subtract(*np.percentile(data, [75, 25]))
                med = np.median(data)
                if (med - 1.5*iqr > x) or (med + 1.5*iqr < x) or abs(x - np.mean(data)) > 2 * np.std(data):
                    filtered.append(med)
                    # filtered.append(np.median(data[max(0, i-5):i] + data[i+1:min(len(data), i+5)]))
                else:
                    filtered.append(x)
                    
            self.data[key] = filtered
        return sampleBad

In [5]:
# {personNum : {state: [sampleNums]}}
# 0 = key for throwing away all samples of that state

badSamples = {
    1: {"active": [5], "neutral": [2], "meditate": []},
    2: {"active": [0], "neutral": [0], "meditate": [0]},
    3: {"active": [1, 4], "neutral": [1], "meditate": [5, 6, 7, 8]},
    4: {"active": [2], "neutral": [7], "meditate": [1, 8]}, # maybe n1
    5: {"active": [], "neutral": [], "meditate": []}, # i love you person 5 
    6: {"active": [], "neutral": [2, 6], "meditate": []},
    7: {"active": [5], "neutral": [4, 6, 7], "meditate": [1, 3, 4, 8]}, # think about killing some of this data
    8: {"active": [5], "neutral": [1], "meditate": []}, # maybe m5 and m8
    9: {"active": [], "neutral": [], "meditate": []}, 
    10: {"active": [6, 8], "neutral": [4, 5, 6], "meditate": []},
    11: {"active": [4], "neutral": [4, 8], "meditate": [1, 2, 3, 5, 7]},
    12: {"active": [2, 3, 8], "neutral": [0], "meditate": [6]}, # maybe n0
    13: {"active": [], "neutral": [8], "meditate": []},
    14: {"active": [4, 5, 8], "neutral": [0], "meditate": [1, 2, 8]}
}

In [6]:
data = []
dataLabels = []

def transcribeFileToSample(personN: int, sampleN: int, state: str):
    sample_data = Sample()

    with open("data/all_data/" + state + "_" + str(personN) + "_" + str(sampleN) + ".csv") as f:
        reader = csv.reader(f)

        header = next(reader)
        
        for row in reader:
            sample_data.recordDataLine(row)
            
        if (0 not in badSamples[personN][state] and sampleN not in badSamples[personN][state]):

            for key in sample_data.data:
                sample_data.data[key] = sample_data.data[key][begin:end]

            sample_data.filter_outliers()
            data.append(sample_data)
            dataLabels.append(state)

for person in range(num_people):
    for state in count_samples:
        for i in range(count_samples[state]):
            transcribeFileToSample(person + 1, i + 1, state)

In [11]:
dataExtracted = []

def safety_check(x):
    if math.isnan(x): return 0
    if math.isinf(x): return 99999999999
    return x

for point in data:
    extractedPoint = []

    for key in point.data:
        if key == 'RawEEG' or key == 'Meditation' or key == 'Attention': continue
        for func in [np.mean, np.std, ant.sample_entropy, ant.petrosian_fd]:
            extractedPoint.append(safety_check(func(point.get(key))))

    extractedPoint.append(safety_check(ant.spectral_entropy(point.getEEG(), sf=1)))
    
    dataExtracted.append(extractedPoint)

C:\Users\mikad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\antropy\entropy.py:249: RuntimeWarning: divide by zero encountered in log2
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
C:\Users\mikad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\antropy\entropy.py:249: RuntimeWarning: invalid value encountered in multiply
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)


In [25]:
X = dataExtracted.copy()
y = dataLabels.copy()
feature_names = []
bands = ['Alpha', 'Low Beta', 'High Beta', 'Gamma', 'Theta', 'Delta']
for band in bands:
    feature_names.append("mean " + band)
    feature_names.append("std " + band)
    feature_names.append("sample entropy " + band)
    feature_names.append("petrosian fd " + band)
feature_names.append("spectal entropy")
percentile = SelectKBest(chi2, k=1)
X_new = percentile.fit_transform(X, y)
percentile.get_feature_names_out(feature_names)

array(['sample entropy High Beta'], dtype=object)

mean Alpha 0.03476809090784033
std Alpha 0.028218762076033254
sample entropy Alpha 0.026148812116149625
petrosian fd Alpha 0.01923465262599053
mean Low Beta 0.05026599002537205
std Low Beta 0.038785805953119354
sample entropy Low Beta 0.025528068390750544
petrosian fd Low Beta 0.022894963118998107
mean High Beta 0.06778958694541182
std High Beta 0.07281027958450406
sample entropy High Beta 0.02439117432565899
petrosian fd High Beta 0.01600969344214077
mean Gamma 0.0825852500847713
std Gamma 0.09386470288526677
sample entropy Gamma 0.023776859873572188
petrosian fd Gamma 0.024894408765733785
mean Theta 0.0672575930011168
std Theta 0.04745956751659164
sample entropy Theta 0.025847780564929354
petrosian fd Theta 0.030968357463410215
mean Delta 0.04243862471461758
std Delta 0.04915445415442107
sample entropy Delta 0.03506598040517744
petrosian fd Delta 0.020356075274980623
spectal entropy 0.02948446578344177
